<a href="https://colab.research.google.com/github/rojinadeuja/Data-Processing-Utilities/blob/main/CSV-to-Corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted  at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Corpus
A corpus is a collection of text documents, and corpora is the plural of corpus. So a custom corpus is really just a bunch of text files in a directory, often alongside many other directories of text files.

To train your own vectors, first you'll need to prepare your corpus as a single text file with all words separated by one or more spaces or tabs. If your corpus has multiple documents, the documents (only) should be separated by new line characters.

## Import Modules

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

## Load Dataset

In [23]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/IMDB.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Create Document array using text field

In [24]:
docs_array = df['review']
print("Dimension of the documents array: ", docs_array.shape)

Dimension of the documents array:  (50000,)


## Tokenize the words

In [25]:
# Function for convert a list of sentences to a list of lists containing tokenized words
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+') # Tokenize the words.
    
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [26]:
import nltk
nltk.download('wordnet')
# Convert a list of sentences to a list of lists containing tokenized words
%time docs = docs_preprocessor(docs_array)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
CPU times: user 1min 29s, sys: 1.42 s, total: 1min 31s
Wall time: 1min 31s


In [27]:
print("Length of the 2D Array of Tokenized Documents: ", len(docs))

Length of the 2D Array of Tokenized Documents:  50000


In [28]:
# View tokenized list of words
docs[0][0:6]

['other', 'reviewer', 'mentioned', 'that', 'after', 'watching']

# Option 1: Create corpus with words only

In [29]:
# Combine list of lists into a single list
from itertools import chain
combined_docs = list(chain.from_iterable(docs)) #

In [30]:
# Separate words by whitespace
combined_docs_sep = ' '.join(combined_docs)

In [31]:
# Check if merged
combined_docs_sep[0:100]

'other reviewer mentioned that after watching just episode hooked they right this exactly what happen'

In [32]:
# Open a file in witre mode and write the list of words to file
fo = open("word_corpus.txt", "w")
fo.writelines(combined_docs_sep)

## Option 2: Create corpus with a single document in each line

In [33]:
with open('docs_corpus.txt', 'w') as f:
  # Loop through each document, add separator and then write to file
  for doc in docs:
      doc_sep = ' '.join(doc)
      f.write("%s\n" % doc_sep)